In [1]:
from requests import get
from bs4 import BeautifulSoup
import pprint
import json

In [2]:
#function to scrape content of main table
def scrape(url,output):
    
    #get html and convert to nice object
    html = get(url,stream=True).content
    html = BeautifulSoup(html, "html.parser")
    
    #get name of section we are scraping and create an object for it
    section = html.body["id"]
    output[section] = {} 
    
    main_table = html.find("div", class_="well")
    
    for td in main_table.select("td"):
        if "data-column-name" in td.attrs:
            output[section][td["data-column-name"]] = td.text
    
    return output

In [3]:
#update with value of &fk0 parameter in URL of project you want to scrape
foreignKey = "1"

In [5]:
#get html of first page and convert to nice object
html = get("http://portal.costukraine.org/uad_mfo/PROJECTS.php?hname=dbo_PROJECTS_dbo_PROJECTS_TECH_DETAILS_handler&fk0=" + foreignKey + "&master_viewmode=0",stream=True).content
html = BeautifulSoup(html, "html.parser")

In [6]:
#set up array to store urls for each view of project
urls = []

#get urls of each page
navigation = html.find("ul", class_="nav nav-tabs grid-details-tabs")
for li in navigation.select("li"):
    urls.append(li.a["href"])

In [7]:
#put amendments URL in separate variable (page has different structure)
amendmentsURL = urls.pop()

In [8]:
#set up object for scraped data
data = {}

In [9]:
#scrape summary table (appears on each page, so only do this once)
data["summary"] = {}

summary_table = html.find("div", class_="grid grid-table grid-master js-grid")

for th in summary_table.select("th"):
        data["summary"][th["data-name"]] = ""
        
for td in summary_table.select("td"):
    if "data-column-name" in td.attrs:
        data["summary"][td["data-column-name"]] = td.text

In [10]:
#scrape main table on each page

for url in urls:
    print("scraping " + url)
    data = scrape("http://portal.costukraine.org/uad_mfo/" + url, data)

scraping PROJECTS.php?hname=dbo_PROJECTS_dbo_PROJECTS_TECH_DETAILS_handler&fk0=1&master_viewmode=0
scraping PROJECTS.php?hname=dbo_PROJECTS_dbo_PROJECTS_SUBJECTS_handler&fk0=1&master_viewmode=0
scraping PROJECTS.php?hname=dbo_PROJECTS_dbo_PROJECTS_CUSTOMER_handler&fk0=1&master_viewmode=0
scraping PROJECTS.php?hname=dbo_PROJECTS_dbo_PROJECTS_FINANCING_handler&fk0=1&master_viewmode=0
scraping PROJECTS.php?hname=dbo_PROJECTS_dbo_PROJECTS_PROJECT_ORGANIZATION_handler&fk0=1&master_viewmode=0
scraping PROJECTS.php?hname=dbo_PROJECTS_dbo_PROJECTS_CONTRACTOR_handler&fk0=1&master_viewmode=0
scraping PROJECTS.php?hname=dbo_PROJECTS_dbo_PROJECTS_ENG_SUPERVIZORY_handler&fk0=1&master_viewmode=0
scraping PROJECTS.php?hname=dbo_PROJECTS_dbo_PROJECTS_TECH_SUPERVIZORY_handler&fk0=1&master_viewmode=0


In [12]:
with open("data_" + foreignKey + ".json", "w") as output:
    json.dump(data, output)